In [36]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
from scipy.optimize import minimize
import scipy.linalg
import numpy as np
import scipy.linalg
from scipy.optimize import differential_evolution, basinhopping, shgo

from epsr import EPSR

## Extended parameter shift rule

In [37]:
# test
d=1
r=3
Omegas=np.arange(1, r + 1)
x = np.array([(2 * mu - 1) / (2 * r) * np.pi for mu in range(1, r + 1)])
# x = np.random.uniform(0, np.pi, r)
b=EPSR(x, Omegas, d)
print(b)

d=2
r=2
Omegas=np.arange(1, r + 1)
x = np.array([ mu * (np.pi / r)  for mu in range(0, r + 1)])
# x = np.random.uniform(0, np.pi, r+1)
b=EPSR(x, Omegas, d)
print(b)


[ 2.48803387 -0.33333333  0.17863279]
[-1.5  2.  -0.5]


## Final variance

In [38]:
def L2norm2FinalVar(x, Omegas, d):
    b=EPSR(x, Omegas, d)
    return 0.5*np.sum(np.abs(b)**2)

def L2normFinalVar(x, Omegas, d):
    b=EPSR(x, Omegas, d)
    return np.sqrt(np.sum(np.abs(b)**2))

def L1normFinalVar(x, Omegas, d):
    b=EPSR(x, Omegas, d)
    return np.sum(np.abs(b))


**EXP 1:**


In [39]:
# r = 1
r = 2
# r = 3
# r = 4
# r = 5
# r = 6
# r = 7

# odd order d
d = 1
# d = 3
# d = 5

Omegas = np.arange(1, r+1)

# L2norm2_objective = lambda x:  L2norm2FinalVar(x, Omegas, d)
# L2norm_objective = lambda x:  L2normFinalVar(x, Omegas, d)
L1norm_objective = lambda x:  L1normFinalVar(x, Omegas, d)

# Initial guess for x_mu
x_mu = np.array([(2 * mu - 1) / (2 * r) * np.pi for mu in range(1, r + 1)])
# x_mu_initial = np.random.uniform(0,  np.pi, r)

# Perform optimization
# result = minimize(objective, x_mu_initial, method='BFGS')
# result = basinhopping(objective, x_mu_initial, niter=100)

# Define bounds for optimization (0 to pi for each variable)
bounds = [(0.01, np.pi-0.01) for _ in range(r)]

result = differential_evolution(L1norm_objective, bounds)
# Perform optimization with bounds
# result = minimize(objective, x_mu_initial, method='SLSQP', bounds=bounds)

# Optimal x_mu and corresponding minimized l1 norm
optimal_x_mu = result.x
optimal_x_mu.sort()
minimized_l1norm = result.fun

# Output the results
print("Optimal x_mu:", optimal_x_mu)
print("Minimized l1 norm:", minimized_l1norm)
print("equadistant x_mu:", x_mu)
print("l1 norm when using equadistantx_mu:", L1norm_objective(x_mu))
print(np.isclose(optimal_x_mu, x_mu))
print(np.isclose(minimized_l1norm, L1norm_objective(x_mu)))

Optimal x_mu: [0.78539843 2.35619583]
Minimized l1 norm: 2.0000000000012914
equadistant x_mu: [0.78539816 2.35619449]
l1 norm when using equadistantx_mu: 2.0
[ True  True]
True



**EXP 2:**


In [40]:
# r = 1
r = 2
# r = 3
# r = 4
# r = 5
# r = 6
# r = 7

# even order d
# d = 2
# d = 4
d = 6

Omegas = np.arange(1, r+1)

L1norm_objective = lambda x:  L1normFinalVar(x, Omegas, d)

# Initial guess for x_mu
x_mu = np.array([ mu * (np.pi / r)  for mu in range(0, r + 1)])

# Define bounds for optimization (0 to pi for each variable)
bounds = [(0, np.pi) for _ in range(r+1)]
# x_mu_initial = np.random.uniform(0,  np.pi, r)

result = differential_evolution(L1norm_objective, bounds)
# Perform optimization with bounds
# result = minimize(objective, x_mu_initial, method='SLSQP', bounds=bounds)

# Optimal x_mu and corresponding minimized l1 norm
optimal_x_mu = result.x
optimal_x_mu.sort()
minimized_l1norm = result.fun

# Output the results
print("Optimal x_mu:", optimal_x_mu)
print("Minimized l1 norm:", minimized_l1norm)
print("equadistant x_mu:", x_mu)
print("l1 norm when using equadistantx_mu:", L1norm_objective(x_mu))
print(np.isclose(optimal_x_mu, x_mu))
print(np.isclose(minimized_l1norm, L1norm_objective(x_mu)))

Optimal x_mu: [0.         1.57079632 3.14159241]
Minimized l1 norm: 64.00000000000188
equadistant x_mu: [0.         1.57079633 3.14159265]
l1 norm when using equadistantx_mu: 64.0
[ True  True  True]
True


**EXP 3: l1 norm_objective sugradient**

In [41]:
# r = 1
# r = 2
# r = 3
# r = 4
r = 5
# r = 6
# r = 7

# odd order d
d = 1
# d = 3
d = 5

Omegas = np.arange(1, r + 1)

# L2norm2_objective = lambda x: L2norm2FinalVar(x, Omegas, d)
# L2norm_objective = lambda x: L2normFinalVar(x, Omegas, d)
L1norm_objective = lambda x: L1normFinalVar(x, Omegas, d)

# Define the analytical subgradient function (just one of the subgradients)
def analytical_subgradient_l1_odd(x, Omegas):
    p = Omegas ** d * (-1 if d % 4 == 3 else 1)
    A = np.sin(np.outer(x, Omegas))
    A_1 = np.cos(np.outer(x, Omegas)) * Omegas
    A_inv = np.linalg.inv(A)
    b = np.linalg.inv(A.T) @ p
    subgrad = - np.diag(A_1 @ A_inv @ np.outer(np.sign(b), b))
    return subgrad

x_mu = np.array([(2 * mu - 1) / (2 * r) * np.pi for mu in range(1, r + 1)])
# x_mu = np.random.uniform(0, np.pi, size=r)
analytical_subgradient_l1_odd(x_mu, Omegas)


array([-4.26325641e-13, -2.27373675e-13, -5.68434189e-13, -9.09494702e-13,
       -8.52651283e-13])

In [42]:
r = 1
r = 2
r = 3
r = 4
r = 5
r = 6
r = 7

# even order d
d = 2
d = 4
d = 6

Omegas = np.arange(1, r + 1)

# L2norm2_objective = lambda x: L2norm2FinalVar(x, Omegas, d)
# L2norm_objective = lambda x: L2normFinalVar(x, Omegas, d)
L1norm_objective = lambda x: L1normFinalVar(x, Omegas, d)

# Define the analytical subgradient function (just one of the subgradients)
def analytical_subgradient_l1_even(x, Omegas):
    q = Omegas ** d * (-1 if d % 4 == 2 else 1)
    q = np.insert(q, 0, 1 if d == 0 else 0)
    Omegas = np.insert(Omegas, 0, 0)
    A = np.cos(np.outer(x, Omegas))
    A_1 = -np.sin(np.outer(x, Omegas)) * Omegas
    A_inv = np.linalg.inv(A)
    b = np.linalg.inv(A.T) @ q
    subgrad = - np.diag(A_1 @ A_inv @ np.outer(np.sign(b), b))
    return subgrad

x_mu = np.array([mu * (np.pi / r) for mu in range(0, r + 1)])
# x_mu = np.random.uniform(0, np.pi, size=r+1)
analytical_subgradient_l1_even(x_mu, Omegas)


array([-0.00000000e+00, -3.09228199e-11, -4.18367563e-11, -2.81943358e-11,
       -3.18323146e-11, -4.18367563e-11, -3.63797881e-11, -2.08856776e-11])